# Previsão de preços com vizinhança

## Neste projeto, criaremos e utilizaremos a variável 'neighborhood' para prever os preços.

In [ ]:
!pip install category_encoders --quiet

### Importação de bibliotecas

In [2]:
import warnings
from glob import glob

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from category_encoders import OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge  # noqa F401
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.utils.validation import check_is_fitted

warnings.simplefilter(action="ignore", category=FutureWarning)

### Preparando os dados

Importando os dados

In [3]:
def wrangle(filepath):
    # Read CSV file
    df = pd.read_csv(filepath)

    # Subset data: Apartments in "Capital Federal", less than 400,000
    mask_ba = df["place_with_parent_names"].str.contains("Capital Federal")
    mask_apt = df["property_type"] == "apartment"
    mask_price = df["price_aprox_usd"] < 400_000
    df = df[mask_ba & mask_apt & mask_price]

    # Subset data: Remove outliers for "surface_covered_in_m2"
    low, high = df["surface_covered_in_m2"].quantile([0.1, 0.9])
    mask_area = df["surface_covered_in_m2"].between(low, high)
    df = df[mask_area]

    # Split "lat-lon" column
    df[["lat", "lon"]] = df["lat-lon"].str.split(",", expand=True).astype(float)
    df.drop(columns="lat-lon", inplace=True)

    # Extract neighborhood
    df["neighborhood"] = df["place_with_parent_names"].str.split("|", expand=True)[3]
    df.drop(columns="place_with_parent_names", inplace=True)

    return df

No projeto anterior, usamos nossa função Wrangle para importar dois arquivos CSV como DataFrames. Mas e se tivéssemos centenas de arquivos CSV para importar? Colocá-los em Wrangle um por um não seria uma opção. Então, vamos começar com uma técnica para ler vários arquivos CSV em um único DataFrame.

O primeiro passo é reunir os nomes de todos os arquivos que queremos importar. Podemos fazer isso usando correspondência de padrões.

In [4]:
files = glob("/content/buenos-aires-real-estate-*.csv")
files

['/content/buenos-aires-real-estate-4.csv',
 '/content/buenos-aires-real-estate-2.csv',
 '/content/buenos-aires-real-estate-5.csv',
 '/content/buenos-aires-real-estate-1.csv',
 '/content/buenos-aires-real-estate-3.csv']

O próximo passo é ler cada um dos CSVs nos arquivos para um DataFrame e colocar todos esses DataFrames em uma lista.

In [5]:
frames = [wrangle(file) for file in files]
print("Total de data frames: ", len(frames))
for frame in frames:
    print(frame.shape)

Total de data frames:  5
(1305, 17)
(1315, 17)
(1331, 17)
(1343, 17)
(1288, 17)


O passo final é usar o pandas para combinar todos os DataFrames em frames.

In [6]:
df = pd.concat(frames, ignore_index=True)
print(df.shape)
df.head()

(6582, 17)


,operation,property_type,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,lat,lon,neighborhood
0,sell,apartment,59000.0,USD,894581.60,59000.0,0.0,40.0,NaN,1475.000000,NaN,2.0,NaN,http://paternal.properati.com.ar/12t3j_venta_d...,-34.604069,-58.460209,Paternal
1,sell,apartment,78900.0,USD,1196313.36,78900.0,43.0,38.0,1834.883721,2076.315789,1.0,1.0,1600.0,http://congreso.properati.com.ar/12j7y_venta_d...,-34.620026,-58.388467,Congreso
2,sell,apartment,240000.0,USD,3638976.00,240000.0,97.0,88.0,2474.226804,2727.272727,NaN,4.0,NaN,http://agronomia.properati.com.ar/10vec_venta_...,-34.599857,-58.489002,Agronomía
3,sell,apartment,75000.0,USD,1137180.00,75000.0,NaN,34.0,NaN,2205.882353,4.0,2.0,NaN,http://balvanera.properati.com.ar/10vnl_venta_...,-34.612338,-58.401328,Balvanera
4,sell,apartment,110000.0,USD,1667864.00,110000.0,NaN,46.0,NaN,2391.304348,2.0,2.0,NaN,http://parque-chas.properati.com.ar/yc23_venta...,-34.586160,-58.475633,Parque Chas


### Explore

Observando a saída da chamada df.head() acima, vemos que precisamos fazer uma pequena limpeza antes de podermos trabalhar com as informações de vizinhança neste conjunto de dados. A boa notícia é que, como estamos usando uma função Wrangle, precisamos apenas alterá-la para limpar novamente todos os nossos arquivos CSV. É por isso que as funções são tão úteis.

Modifiquei a função Wrangle para criar um novo recurso "neighborhood". Essa informação pode ser encontrada a vizinhança de cada propriedade na coluna "place_with_parent_names". Por exemplo, uma propriedade com o nome de lugar "|Argentina|Capital Federal|Palermo|" está localizada na vizinhança "Palermo". Além disso, atualizei a função para remover a coluna "place_with_parent_names".

### Split

In [7]:
target = "price_aprox_usd"
features = ["neighborhood"]
y_train = df[target]
X_train = df[features]

### Build Model

Baseline

In [8]:
y_mean = y_train.mean()
y_pred_baseline = [y_mean] * len(y_train)

print("Mean apt price:", y_mean)
print("Baseline MAE:", mean_absolute_error(y_train, y_pred_baseline))

Mean apt price: 132383.83701458524
Baseline MAE: 44860.10834274133


### Iterate

In [9]:
ohe = OneHotEncoder(use_cat_names=True)
ohe.fit(X_train)
XT_train = ohe.transform(X_train)
print(XT_train.shape)
XT_train.head()

(6582, 57)


,neighborhood_Paternal,neighborhood_Congreso,neighborhood_Agronomía,neighborhood_Balvanera,neighborhood_Parque Chas,neighborhood_Villa Pueyrredón,neighborhood_Palermo,neighborhood_Boedo,neighborhood_Caballito,neighborhood_Coghlan,...,neighborhood_Las Cañitas,neighborhood_San Nicolás,neighborhood_Mataderos,neighborhood_Villa Lugano,neighborhood_Versalles,neighborhood_Villa Ortuzar,neighborhood_Pompeya,neighborhood_Parque Avellaneda,neighborhood_Catalinas,neighborhood_Tribunales
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
model = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    LinearRegression()
)
model.fit(X_train, y_train)

Pipeline(steps=[('onehotencoder',
                 OneHotEncoder(cols=['neighborhood'], use_cat_names=True)),
                ('linearregression', LinearRegression())])